Supporting Functions for extracting payload and converting them to string

In [93]:
import pandas as pd
from sklearn.utils import resample
import ast 
import string
# Function to convert hex values to ASCII string
def hex_to_ascii(hex_list):
    ascii_str = ''

    #print(len(hex_list))
    for hex_val in hex_list:

    # Split each hex value into pairs and convert them to ASCII characters
      # Split each hex value into pairs and convert them to ASCII characters
      ascii_str += ''.join([chr(int(hex_val[i:i+2], 16)) for i in range(0, len(hex_val), 2)])
      filtered_str = ''.join(filter(lambda x: x in string.printable, ascii_str))
    #print(filtered_str)

    return filtered_str


### TFIDF based Example for string based payload NIDS

Reading Some web attacks from CIC-IoT 2023 dataset and undersampling them

In [34]:
cross=pd.read_csv("/scratch/user/syedwali/Datasets/undersampled_iot23_dataset.csv",low_memory=False)

In [35]:
cross['label'].value_counts()

label
DDoS-RSTFINFlood           5000
Mirai-udpplain             5000
DDoS-SynonymousIP_Flood    5000
Recon-PortScan             5000
DoS-UDP_Flood              5000
DDoS-ICMP_Fragmentation    5000
DoS-TCP_Flood              5000
DoS-HTTP_Flood             5000
Recon-HostDiscovery        5000
DDoS-UDP_Fragmentation     5000
DDoS-SYN_Flood             5000
Mirai-greip_flood          5000
DDoS-PSHACK_Flood          5000
benign                     5000
DDoS-TCP_Flood             5000
DNS_Spoofing               5000
DDoS-SlowLoris             5000
MITM-ArpSpoofing           5000
DDoS-ACK_Fragmentation     5000
DoS-SYN_Flood              5000
DDoS-ICMP_Flood            5000
Mirai-greeth_flood         5000
DDoS-HTTP_Flood-           5000
DDoS-UDP_Flood             5000
SqlInjection               2830
DictionaryBruteForce       2184
BrowserHijacking            972
CommandInjection            275
Backdoor_Malware            244
XSS                         222
Name: count, dtype: int64

In [38]:
from sklearn.utils import resample
# Classes to extract and the number of samples per class
classes_to_extract = ['benign', 'XSS', 'SqlInjection']

# Create an empty DataFrame to store the extracted samples
df_extracted = cross[cross['label'].isin(classes_to_extract)]

# Display the value counts for each label after extraction
print(df_extracted['label'].value_counts())



label
benign          5000
SqlInjection    2830
XSS              222
Name: count, dtype: int64


In [39]:
# Use replace to map specific labels to numeric values
df_extracted['label'] = df_extracted['label'].replace({
    'benign': 0,        # Map 'benign' to 0
    'SqlInjection': 1,  # Map 'SqlInjection' to 1
    'XSS': 2            # Map 'XSS' to 2
})

# Display the new label values after conversion to numeric
print(df_extracted['label'].value_counts())


label
0    5000
1    2830
2     222
Name: count, dtype: int64


/tmp/job.11542174/ipykernel_149109/2119393291.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['label'] = df_extracted['label'].replace({


In [40]:
df_extracted['udps.payload_data'] = df_extracted['udps.payload_data'].apply(ast.literal_eval)
df_extracted['string'] = df_extracted['udps.payload_data'].apply(hex_to_ascii)

/tmp/job.11542174/ipykernel_149109/4075109190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['udps.payload_data'] = df_extracted['udps.payload_data'].apply(ast.literal_eval)
/tmp/job.11542174/ipykernel_149109/4075109190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['string'] = df_extracted['udps.payload_data'].apply(hex_to_ascii)


In [41]:
# Find rows where X_train['string'] is empty or contains only whitespace characters
empty_rows = df_extracted['string'].str.strip().eq('')
# Get the indices of the rows to be dropped
indices_to_drop = empty_rows[empty_rows].index
# Drop the rows from the DataFrame
df_extracted = df_extracted.drop(indices_to_drop)
# Reset the index after dropping rows
df_extracted.reset_index(drop=True, inplace=True)
# Check the modified DataFrame
print(len(indices_to_drop))

330


In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming 'data' contains the 'string' and 'label' columns
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_extracted['string'], df_extracted['label'], test_size=0.1, random_state=42)

# Initialize the TF-IDF vectorizer
tfidf = TfidfVectorizer()

# Fit the vectorizer on the training data and transform both train and test data
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Initialize the Random Forest Classifier with parallel processing (n_jobs=-1)
rf_classifier = RandomForestClassifier(n_jobs=-1, random_state=42)

# Train the classifier on the TF-IDF features and labels
rf_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9728331177231565
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       482
           1       1.00      0.94      0.97       269
           2       0.94      0.73      0.82        22

    accuracy                           0.97       773
   macro avg       0.97      0.89      0.92       773
weighted avg       0.97      0.97      0.97       773



Reading SQL and benign from CIC-IDS 2017 dataset for cross validation 

In [73]:
import pandas as pd
cic_17=pd.read_csv('/scratch/user/syedwali/Datasets/CIC_2017/processed/split_files/labeled_csv/thur_processed.1.csv',low_memory=False)

In [63]:
cic_17['label'].value_counts()

label
BENIGN                        96170
Web Attack  XSS               1459
Web Attack  Brute Force        413
Web Attack  Sql Injection       15
Name: count, dtype: int64

In [74]:
import pandas as pd
from sklearn.utils import resample

# Create an empty DataFrame to store the undersampled data
df_undersampled = pd.DataFrame()

# Set the number of BENIGN samples to keep
benign_samples = 20

# Iterate over each class
for label in cic_17['label'].unique():
    # Filter the DataFrame for the current class
    df_class = cic_17[cic_17['label'] == label]
    
    # If the class is 'BENIGN', undersample it to 1000 samples
    if label == 'BENIGN':
        df_class = resample(df_class, replace=False, n_samples=benign_samples, random_state=42)
    
    # Append the class DataFrame to the undersampled DataFrame
    df_undersampled = pd.concat([df_undersampled, df_class])

# Shuffle the undersampled dataset to avoid any ordering bias
df_undersampled = df_undersampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the value counts for each label after undersampling
print(df_undersampled['label'].value_counts())


label
Web Attack  XSS              1459
Web Attack  Brute Force       413
BENIGN                          20
Web Attack  Sql Injection      15
Name: count, dtype: int64


In [75]:
filtered_classes=['Web Attack \x96 Sql Injection','BENIGN']
df_undersampled=df_undersampled[df_undersampled['label'].isin(filtered_classes)]

In [76]:
df_undersampled['label'].value_counts()

label
BENIGN                        20
Web Attack  Sql Injection    15
Name: count, dtype: int64

In [77]:
# Use replace to map specific labels to numeric values
df_undersampled['label'] = df_undersampled['label'].replace({
    'BENIGN': 0,        # Map 'benign' to 0
    'Web Attack \x96 Sql Injection': 1,  # Map 'SqlInjection' to 1
   # 'Web Attack \x96 XSS': 2            # Map 'XSS' to 2
})

# Display the new label values after conversion to numeric
print(df_undersampled['label'].value_counts())

label
0    20
1    15
Name: count, dtype: int64


In [78]:
import ast
# Convert each string representation to a list
df_undersampled['udps.payload_data'] = df_undersampled['udps.payload_data'].apply(ast.literal_eval)

In [79]:
df_undersampled['string'] = df_undersampled['udps.payload_data'].apply(hex_to_ascii)

In [80]:
# Find rows where X_train['string'] is empty or contains only whitespace characters
empty_rows = df_undersampled['string'].str.strip().eq('')

# Get the indices of the rows to be dropped
indices_to_drop = empty_rows[empty_rows].index

# Drop the rows from the DataFrame
df_undersampled = df_undersampled.drop(indices_to_drop)

# Reset the index after dropping rows
df_undersampled.reset_index(drop=True, inplace=True)

# Check the modified DataFrame
print(len(indices_to_drop))

13


In [81]:
X_test_tfidf = tfidf.transform(df_undersampled['string'])

In [82]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_tfidf)
y_test=df_undersampled['label']
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8636363636363636
Classification Report:
              precision    recall  f1-score   support

           0       0.77      1.00      0.87        10
           1       1.00      0.75      0.86        12

    accuracy                           0.86        22
   macro avg       0.88      0.88      0.86        22
weighted avg       0.90      0.86      0.86        22



### Training and testing Flow-based NIDS for comparison

Now Preparing Dataset for flow based NIDS by dropping irrelevant columns

In [83]:
for i in df_extracted.columns:
    print(i)

id
expiration_id
src_ip
src_mac
src_oui
src_port
dst_ip
dst_mac
dst_oui
dst_port
protocol
ip_version
vlan_id
tunnel_id
bidirectional_first_seen_ms
bidirectional_last_seen_ms
bidirectional_duration_ms
bidirectional_packets
bidirectional_bytes
src2dst_first_seen_ms
src2dst_last_seen_ms
src2dst_duration_ms
src2dst_packets
src2dst_bytes
dst2src_first_seen_ms
dst2src_last_seen_ms
dst2src_duration_ms
dst2src_packets
dst2src_bytes
bidirectional_min_ps
bidirectional_mean_ps
bidirectional_stddev_ps
bidirectional_max_ps
src2dst_min_ps
src2dst_mean_ps
src2dst_stddev_ps
src2dst_max_ps
dst2src_min_ps
dst2src_mean_ps
dst2src_stddev_ps
dst2src_max_ps
bidirectional_min_piat_ms
bidirectional_mean_piat_ms
bidirectional_stddev_piat_ms
bidirectional_max_piat_ms
src2dst_min_piat_ms
src2dst_mean_piat_ms
src2dst_stddev_piat_ms
src2dst_max_piat_ms
dst2src_min_piat_ms
dst2src_mean_piat_ms
dst2src_stddev_piat_ms
dst2src_max_piat_ms
bidirectional_syn_packets
bidirectional_cwr_packets
bidirectional_ece_packets
bi

In [84]:
df_extracted.drop(['id','expiration_id','src_ip','src_mac','src_oui','src_port','dst_ip','dst_mac','dst_oui','dst_port',
'protocol','ip_version','vlan_id','tunnel_id','bidirectional_first_seen_ms','bidirectional_last_seen_ms','src2dst_first_seen_ms',
'src2dst_last_seen_ms','dst2src_first_seen_ms','dst2src_last_seen_ms','application_name','application_category_name',
'application_is_guessed','application_confidence','requested_server_name','client_fingerprint','server_fingerprint','user_agent',
'content_type','udps.payload_data','udps.delta_time','udps.packet_direction','udps.ip_size','udps.transport_size','udps.payload_size',
'udps.syn','udps.cwr','udps.ece','udps.urg','udps.ack','udps.psh','udps.rst','udps.fin','file'],axis=1,inplace=True)

In [86]:
# X will contain all features except the label columns
X = df_extracted.drop(['label','string'], axis=1)
# Y will be the encoded labels
y = df_extracted['label']
# Step 2: Split into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [88]:
from sklearn.ensemble import RandomForestClassifier
# Train a Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=42,n_jobs=-1)
rf_clf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_clf.predict(X_test)

# Evaluate the classifier
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Classification Report for Random Forest:\n{classification_report(y_test, y_pred_rf)}")

Random Forest Accuracy: 0.9914
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1411
           1       1.00      0.98      0.99       842
           2       0.90      0.95      0.92        64

    accuracy                           0.99      2317
   macro avg       0.96      0.98      0.97      2317
weighted avg       0.99      0.99      0.99      2317



Now Preparing CIC-IDS 2017 dataset for cross validation purpose

In [89]:
for i in df_undersampled.columns:
    print(i)

id
expiration_id
src_ip
src_mac
src_oui
src_port
dst_ip
dst_mac
dst_oui
dst_port
protocol
ip_version
vlan_id
tunnel_id
bidirectional_first_seen_ms
bidirectional_last_seen_ms
bidirectional_duration_ms
bidirectional_packets
bidirectional_bytes
src2dst_first_seen_ms
src2dst_last_seen_ms
src2dst_duration_ms
src2dst_packets
src2dst_bytes
dst2src_first_seen_ms
dst2src_last_seen_ms
dst2src_duration_ms
dst2src_packets
dst2src_bytes
bidirectional_min_ps
bidirectional_mean_ps
bidirectional_stddev_ps
bidirectional_max_ps
src2dst_min_ps
src2dst_mean_ps
src2dst_stddev_ps
src2dst_max_ps
dst2src_min_ps
dst2src_mean_ps
dst2src_stddev_ps
dst2src_max_ps
bidirectional_min_piat_ms
bidirectional_mean_piat_ms
bidirectional_stddev_piat_ms
bidirectional_max_piat_ms
src2dst_min_piat_ms
src2dst_mean_piat_ms
src2dst_stddev_piat_ms
src2dst_max_piat_ms
dst2src_min_piat_ms
dst2src_mean_piat_ms
dst2src_stddev_piat_ms
dst2src_max_piat_ms
bidirectional_syn_packets
bidirectional_cwr_packets
bidirectional_ece_packets
bi

In [91]:
df_undersampled.drop(['id','expiration_id','src_ip','src_mac','src_oui','src_port','dst_ip','dst_mac','dst_oui','dst_port',
'protocol','ip_version','vlan_id','tunnel_id','bidirectional_first_seen_ms','bidirectional_last_seen_ms','src2dst_first_seen_ms',
'src2dst_last_seen_ms','dst2src_first_seen_ms','dst2src_last_seen_ms','application_name','application_category_name',
'application_is_guessed','application_confidence','requested_server_name','client_fingerprint','server_fingerprint','user_agent',
'content_type','udps.payload_data','udps.delta_time','udps.packet_direction','udps.ip_size','udps.transport_size','udps.payload_size',
'udps.syn','udps.cwr','udps.ece','udps.urg','udps.ack','udps.psh','udps.rst','udps.fin','flowid','string'],axis=1,inplace=True)

In [92]:
# X will contain all features except the label columns
X_test = df_undersampled.drop(['label'], axis=1)

# Y will be the encoded labels
y_test = df_undersampled['label']

# Make predictions
y_pred_rf = rf_clf.predict(X_test)

# Evaluate the classifier
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Classification Report for Random Forest:\n{classification_report(y_test, y_pred_rf)}")

Random Forest Accuracy: 0.4545
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.45      1.00      0.62        10
           1       0.00      0.00      0.00        12

    accuracy                           0.45        22
   macro avg       0.23      0.50      0.31        22
weighted avg       0.21      0.45      0.28        22



/scratch/user/syedwali/Python_env/Processing/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/user/syedwali/Python_env/Processing/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/user/syedwali/Python_env/Processing/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,